# <b><font color='#009e74'>Rapid protein stability prediction using deep learning representations </font></b>

Preprint pipeline version for predicting protein variants **thermodynamic stability changes** ($\Delta \Delta G$) using a deep learning representation. The program, using as input a protein structure (uploaded as PDB) returns stability predictions ($\Delta \Delta G$ in kcal/mol) for each variant at each position of the query protein.
More details can be found in: **Blaabjerg et al.:** ["Rapid protein stability prediction using deep learning representations"](https://www.biorxiv.org/content/10.1101/2022.07.14.500157v1). Source code is available on the project [Github](https://github.com/KULL-Centre/papers/tree/main/2022/ML-ddG-Blaabjerg-et-al) page.



##  <b><font color='#009e74'> Reminders and Important informations:</font></b>
- This notebook  <b><font color='#d55c00'>must</font></b> be run in a Colab GPU session (go to page menu: `Runtime`->  `Change runtime type` -> select `GPU` and confirm
- Cells named as  <b><font color='#56b4e9'>PRELIMINARY OPERATIONS </font></b> have to be run <b><font color='#d55c00'>ONCE only at the start</font></b>  and  skipped for new predictions.
- <b><font color='#d55c00'>ONE</font></b> single pdb at the time can be processed by the pipeline. 
- A  <b><font color='#d55c00'>new run</font></b> can be perform input direcly the new structure in the pdb upload cell and run the prediction cell again

****

## <b><font color='#009e74'>PIPELINE : PRELIMINARY OPERATIONS </font></b>
These cells should be run once at the start of the notebook
****

In [ ]:
#@title <b><font color='#56b4e9'>PRELIMINARY OPERATIONS</font>: Setup enviroment and dependencies</b>

#@markdown Run this cell to install the required enviroment and dependencies

#@markdown **N.B: This cell should be run only ONCE at the START of the notebook.**
%%bash
rm -r sample_data

#install minconda
env PYTHONPATH= &> /dev/null
MINICONDA_INSTALLER_SCRIPT=Miniconda3-py37_4.9.2-Linux-x86_64.sh &> /dev/null
MINICONDA_PREFIX=/usr/local
wget https://repo.continuum.io/miniconda/$MINICONDA_INSTALLER_SCRIPT &> /dev/null
chmod +x $MINICONDA_INSTALLER_SCRIPT &> /dev/null
./$MINICONDA_INSTALLER_SCRIPT -b -f -p $MINICONDA_PREFIX  &> /dev/null

rm /content/Miniconda3-py37_4.9.2-Linux-x86_64.sh

#update miniconda to latest 3.7
conda install --channel defaults conda python=3.7 --yes &> /dev/null
conda update --channel defaults --all --yes &> /dev/null

echo "-> Anaconda installed!"

#install dependencies

# install dependencies present in pip
pip install numpy==1.17.3 torch==1.2.0 biopython==1.72 matplotlib==3.1.1 pdb-tools &> /dev/null
pip install --upgrade pdb-tools &> /dev/null

# install remaining dependencies with conda
conda install  mpl-scatter-density ptitprince pdbfixer=1.5 openmm=7.3.1 -c omnia -c conda-forge -c anaconda -c defaults --yes &> /dev/null

echo "--> Extra python packages installed!"

#install svn
apt-get install -qq subversion &> /dev/null

#mkdir of necessary folders
mkdir data
mkdir data/test
mkdir data/test/predictions
mkdir data/test/predictions/raw
mkdir data/test/predictions/cleaned
mkdir data/test/predictions/parsed
mkdir output/
mkdir output/predictions

#download project folders from github

svn checkout https://github.com/KULL-Centre/papers/trunk/2022/ML-ddG-Blaabjerg-et-al/src  &> /dev/null
svn checkout https://github.com/KULL-Centre/papers/trunk/2022/ML-ddG-Blaabjerg-et-al/output/cavity_models  &> /dev/null
svn checkout https://github.com/KULL-Centre/papers/trunk/2022/ML-ddG-Blaabjerg-et-al/output/ds_models  &> /dev/null

wget -cq https://github.com/KULL-Centre/papers/raw/papers/2022/ML-ddG-Blaabjerg-et-al/data/pdb_frequencies.npz -o /content/data/pdb_frequencies.npz
wget -cq https://github.com/KULL-Centre/papers/raw/main/2022/ML-ddG-Blaabjerg-et-al/colab_additonals/colab_additional.zip

#extra files for runnin the notebooks

mv ds_models ./output/
mv cavity_models ./output/

unzip colab_additional.zip &> /dev/null
rm colab_additional.zip

mv /content/colab_additional/best_model_path.txt /content/output/cavity_models/
mv /content/colab_additional/clean_pdb.py /content/src/pdb_parser_scripts/
mv /content/colab_additional/helpers.py /content/src/
mv /content/colab_additional/pdb_frequencies.npz /content/data/

echo "---> Github data imported!"

#get and compile reduce

cd src/pdb_parser_scripts
git clone https://github.com/rlabduke/reduce.git
cd reduce/
make &> /dev/null

mv /content/colab_additional/reduce /content/src/pdb_parser_scripts/reduce/

chmod +x /content/src/pdb_parser_scripts/reduce/reduce 
echo "----> reduce installed"

rm -r /content/colab_additional
#@markdown ****

-> Anaconda installed!
--> Extra python packages installed!
---> Github data imported!
----> reduce installed


Cloning into 'reduce'...


In [ ]:
#@title <b><font color='#56b4e9'>PRELIMINARY OPERATIONS</font>: Import python libraries, functions and setup common variables</b>

#@markdown Run this cell to import libraries and functions necessary for the pipeline.

#@markdown **N.B: This cell should be run only ONCE at the START of the notebook.**

import sys
sys.path.append("/usr/local/lib/python3.7/site-packages")

import glob
import os
import random
import pathlib
import subprocess
import numpy as np
import pandas as pd
import torch
import time
import datetime
import matplotlib
from pdbfixer import PDBFixer
from simtk.openmm.app import PDBFile
from Bio.PDB.Polypeptide import index_to_one, one_to_index
from collections import OrderedDict
from torch.utils.data import DataLoader, Dataset
from google.colab import files

sys.path.append('./src/')

from cavity_model import (
     CavityModel,
     DownstreamModel,
     ResidueEnvironment,
     ResidueEnvironmentsDataset,
)

from helpers import (
     populate_dfs_with_resenvs,
     remove_disulfides,
     fermi_transform,
     inverse_fermi_transform,
     init_lin_weights,
     ds_pred,
     cavity_to_prism,
     get_seq_from_variant,
)

from visualization import (
     hist_plot,
)

#Extra function to fix pdb

# Setup pipeline parameters
## Set seeds
np.random.seed(0)
random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)
torch.backends.cudnn.enabled = False
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

## Main deep parameters
DEVICE = "cuda"  # "cpu" or "cuda"
NUM_ENSEMBLE = 10
TASK_ID = int(1)
PER_TASK = int(1)

#@markdown ****

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pyrosettacolabsetup

import os
import pandas as pd
# levinthal / paradox
import pyrosettacolabsetup; pyrosettacolabsetup.install_pyrosetta()
import pyrosetta; pyrosetta.init()
import pyrosetta
pyrosetta.init()
from pyrosetta.teaching import *

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Mounted at /content/google_drive
Looking for compatible PyRosetta wheel file at google-drive/PyRosetta/colab.bin/wheels...
Found compatible wheel: /content/google_drive/MyDrive/PyRosetta/colab.bin/wheels//content/google_drive/MyDrive/PyRosetta/colab.bin/wheels/pyrosetta-2022.47+release.d2aee95a6b7-cp37-cp37m-linux_x86_64.whl
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


PyRosetta-4 2022 [Rosetta PyRosetta4.MinSizeRel.python37.ubuntu 2022.47+release.d2aee95a6b7bf6ee70c5e2c7b29d0915e9112fa7 2022-11-23T13:33:36] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init: Rosetta version: PyRosetta4.MinSizeRel.python37.ubuntu r336 2022.47+release.d2aee95a6b7 d2aee95a6b7bf6ee70c5e2c

In [3]:
from pyrosetta.rosetta.protocols.ddg.CartesianddG import MutationSet

In [11]:
pdb_path = '/content/google_drive/MyDrive/kaggle_deep_learning/final_project/pdbs/'
mut = [pdb_path + i for i in os.listdir(pdb_path) if i.endswith('.pdb')][0]
wt = '/content/google_drive/MyDrive/kaggle_deep_learning/final_project/cd1a.pdb'
wt_pose = pyrosetta.pose_from_pdb(wt)
mt_pose = pyrosetta.pose_from_pdb(mut)
mt_set = MutationSet([17], ['A'], 3)
# mt_set.add_wildtypes(wt_pose)
# pyrosetta.rosetta.protocols.ddg.CartesianddG.run(

core.import_pose.import_pose: File '/content/google_drive/MyDrive/kaggle_deep_learning/final_project/cd1a.pdb' automatically determined to be of type PDB
core.conformation.Conformation: [ WARNING ] missing heavyatom:  OXT on residue LYS:CtermProteinFull 221
core.conformation.Conformation: Found disulfide between residues 47 177
core.conformation.Conformation: current variant for 47 CYS
core.conformation.Conformation: current variant for 177 CYS
core.conformation.Conformation: current variant for 47 CYD
core.conformation.Conformation: current variant for 177 CYD
core.conformation.Conformation: Found disulfide between residues 52 188
core.conformation.Conformation: current variant for 52 CYS
core.conformation.Conformation: current variant for 188 CYS
core.conformation.Conformation: current variant for 52 CYD
core.conformation.Conformation: current variant for 188 CYD
core.import_pose.import_pose: File '/content/google_drive/MyDrive/kaggle_deep_learning/final_project/pdbs/33303.pdb' autom

TypeError: ignored

In [15]:
pyrosetta.rosetta.utility.vector1_unsigned_long([])

TypeError: ignored

In [ ]:
from src.cavity_model import *

pdbs = os.listdir('/content/drive/MyDrive/kaggle_deep_learning/final_project/pdbs/')
pdbs = [i for i in pdbs if i.endswith('.pdb')]
for pdb in pdbs[:5]:
  in_clean = '/content/drive/MyDrive/kaggle_deep_learning/final_project/pdbs/' + pdb
  out_clean = '/content/drive/MyDrive/kaggle_deep_learning/final_project/cleaned/'
  in_parse = '/content/drive/MyDrive/kaggle_deep_learning/final_project/cleaned/' + pdb.split('.')[0] + '_clean.pdb'
  out_parse = '/content/drive/MyDrive/kaggle_deep_learning/final_project/parsed/'
  !python3 /content/src/pdb_parser_scripts/clean_pdb.py --pdb_file_in "$in_clean" --out_dir "$out_clean" --reduce_exe /content/src/pdb_parser_scripts/reduce/reduce #&> /dev/null
  !python3 /content/src/pdb_parser_scripts/extract_environments.py --pdb_in "$in_parse" --out_dir "$out_parse"


        if warning inappropriate, adjust the cuttoffs using -METALBump or -NONMETALBump.
        if warning inappropriate, adjust the cuttoffs using -METALBump or -NONMETALBump.
        if warning inappropriate, adjust the cuttoffs using -METALBump or -NONMETALBump.
        if warning inappropriate, adjust the cuttoffs using -METALBump or -NONMETALBump.
Time for cleaning /content/drive/MyDrive/kaggle_deep_learning/final_project/pdbs/33303.pdb: 6.872405052185059
/usr/local/lib/python3.7/site-packages/Bio/PDB/Vector.py:42: BiopythonDeprecationWarning: The module Bio.PDB.Vector has been deprecated in favor of new module Bio.PDB.vectors to solve a name collision with the class Vector. For the class Vector, and vector functions like calc_angle, import from Bio.PDB instead.
  "import from Bio.PDB instead.", BiopythonDeprecationWarning)
/content/src/pdb_parser_scripts/extract_environments.py:109: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning

In [ ]:
from src.helpers import *

parsed_pdb_filenames = os.listdir(out_parse)
parsed_pdb_filenames = [out_parse + pdb for pdb in parsed_pdb_filenames]
BATCH_SIZE = 100
TRAIN_VAL_SPLIT = 0.77
DEVICE = "cuda"
populate_dfs_with_resenvs(ddg_data, resenv_dataset)
dataloader_train = get_ddg_dataloader(ddg_data, 'train', BATCH_SIZE, DEVICE)
dataloader_val = get_ddg_dataloader(ddg_data, 'val', BATCH_SIZE, DEVICE)
df_ddg = None
best_cavity_model_path = open(f"/content/output/cavity_models/best_model_path.txt", "r").read()
cavity_model_net = CavityModel(get_latent=True).to(DEVICE)
cavity_model_net.load_state_dict(torch.load(f"{best_cavity_model_path}"))
loss_func = torch.nn.L1Loss()
EPOCHS = 1
for idx in [range(NUM_ENSEMBLE)[0]]:
  ds_model_net = DownstreamModel().to(DEVICE)
  ds_model_net.load_state_dict(
                  torch.load(
                      f"/content/output/ds_models/ds_model_{idx}/model.pt"
                  )
              )
  optimizer = torch.optim.Adam(ds_model_net.parameters(), lr=0.0003)
  model_idx = idx + NUM_ENSEMBLE
  ds_train_val(
      df_ddg,
      dataloader_train,
      dataloader_val,
      cavity_model_net,
      ds_model_net,
      loss_func,
      optimizer,
      model_idx,
      EPOCHS,
      DEVICE,
  )

Training data set includes 3 pdbs with 663 environments.
Validation data set includes 2 pdbs with 442 environments.
Epoch: 1/1


ValueError: ignored

## <b><font color='#009e74'>PIPELINE : PREDICTIONS </font></b>

In [ ]:
#@title <b><font color='#56b4e9'> PDB upload</font></b>

#@markdown Choose between <b><font color='#d55c00'> ONE</font></b> of the possible input sources for the target pdb and <ins>leave the other cells empty or unmarked</ins>
#@markdown - AlphaFold2 PDB via Uniprot ID:
AF_ID ='P04036'#@param {type:"string"}
#@markdown - PDB ID (imported from RCSB PDB):
PDB_ID =''#@param {type:"string"}
#@markdown - Upload custom PDB
PDB_custom =False#@param {type:"boolean"}

#@markdown

#@markdown Select target chain (default A)
chain='A' #@param {type:'string'}

if os.path.exists("/content/query_protein.pdb"):
    os.remove("/content/query_protein.pdb")
if os.path.exists("/content/data/test/predictions/raw/query_protein_uniquechain.pdb"):
    os.remove("/content/data/test/predictions/raw/query_protein_uniquechain.pdb")
if os.path.exists("/content/data/test/predictions/cleaned/query_protein_uniquechain_clean.pdb"):
    os.remove("/content/data/test/predictions/cleaned/query_protein_uniquechain_clean.pdb")
if os.path.exists("/content/data/test/predictions/parsed/query_protein_uniquechain_clean_coordinate_features.npz"):
    os.remove("/content/data/test/predictions/parsed/query_protein_uniquechain_clean_coordinate_features.npz")

if (AF_ID !='') and (len(AF_ID)==6) : 
    subprocess.call(['curl','-s','-f',f'https://alphafold.ebi.ac.uk/files/AF-{AF_ID}-F1-model_v2.pdb','-o','/content/query_protein.pdb'])
elif (PDB_ID !='') and (len(PDB_ID)==4):
    subprocess.call(['curl','-s','-f',f'https://files.rcsb.org/download/{PDB_ID}.pdb','-o','/content/query_protein.pdb'])

elif PDB_custom:
  print('Upload PDB file:')
  uploaded_pdb = files.upload()
  for fn in uploaded_pdb.keys():
    os.rename(fn, f"/content/query_protein.pdb")
    print('PDB file correctly loaded')

else:
  print(f'ERROR: any PDB uploaded, please select one of the above inputs')



#@markdown N.B. This cell will also perform preliminary operations to correcly format the uploaded PDB

## remove other chains and move to raw folder
!pdb_selchain -"$chain" /content/query_protein.pdb | pdb_delhetatm | pdb_delres --999:0:1 | pdb_fixinsert | pdb_tidy  > /content/data/test/predictions/raw/query_protein_uniquechain.pdb
# Select PDBs to run during this task - could be simplified if we decide to set PER_TASK = 1 for all cases

pdb_input_dir = "data/test/predictions/raw/"
input_pdbs = sorted(list(filter(lambda x: x.endswith(".pdb"), os.listdir('data/test/predictions/raw/'))))
start = (TASK_ID-1)*(PER_TASK)
end = (TASK_ID*PER_TASK)
if end > len(input_pdbs):
    end = len(input_pdbs) #avoid end index exceeding length of list
pdbs = input_pdbs[start:end] 
pdb_names = [i.split(".")[0] for i in pdbs]
print(pdb_names)
print(f"Pre-processing PDBs ...")

!python3 /content/src/pdb_parser_scripts/clean_pdb.py --pdb_file_in /content/data/test/predictions/raw/query_protein_uniquechain.pdb --out_dir /content/data/test/predictions/cleaned/ --reduce_exe /content/src/pdb_parser_scripts/reduce/reduce #&> /dev/null
!python3 /content/src/pdb_parser_scripts/extract_environments.py --pdb_in /content/data/test/predictions/cleaned/query_protein_uniquechain_clean.pdb  --out_dir /content/data/test/predictions/parsed/  #&> /dev/null

if os.path.exists("/content/data/test/predictions/parsed/query_protein_uniquechain_clean_coordinate_features.npz"):
  print(f"Pre-processing PDBs correctly ended")
else:
  print(f"Pre-processing PDB didn't end correcly, please check input informations")

#@markdown ****

In [ ]:

#@title <b><font color='#56b4e9'> Pipeline RUN </font></b>

#@markdown <b><font color='#d55c00'>Execute the cell</font></b> to run the pipeline and generate **saturation mutagenesis predictions of thermodynamic stability changes** predictions

### Pre-process structure data

# Create temporary residue environment datasets to more easily match ddG data
pdb_filenames_ds = sorted(glob.glob(f"/content/data/test/predictions/parsed/*coord*"))

dataset_structure = ResidueEnvironmentsDataset(pdb_filenames_ds, transformer=None)

resenv_dataset = {}
for resenv in dataset_structure:
    if AF_ID!='':
      key = (f"--{AF_ID}--{resenv.chain_id}--{resenv.pdb_residue_number}--{index_to_one(resenv.restype_index)}--"
          )
    elif PDB_ID!='':
      key = (f"--{PDB_ID}--{resenv.chain_id}--{resenv.pdb_residue_number}--{index_to_one(resenv.restype_index)}--"
          )
    else:
      key = (f"--{'CUSTOM'}--{resenv.chain_id}--{resenv.pdb_residue_number}--{index_to_one(resenv.restype_index)}--"
          )
    resenv_dataset[key] = resenv
df_structure_no_mt = pd.DataFrame.from_dict(resenv_dataset, orient='index', columns=["resenv"])
df_structure_no_mt.reset_index(inplace=True)
df_structure_no_mt["index"]=df_structure_no_mt["index"].astype(str)
res_info = pd.DataFrame(df_structure_no_mt["index"].str.split('--').tolist(),
                        columns = ['blank','pdb_id','chain_id','pos','wt_AA', 'blank2'])

df_structure_no_mt["pdbid"] = res_info['pdb_id']
df_structure_no_mt["chainid"] = res_info['chain_id']
df_structure_no_mt["variant"] = res_info["wt_AA"] + res_info['pos'] + "X"
aa_list = ["A", "C", "D", "E", "F", "G", "H", "I", "K", "L", 
            "M", "N", "P", "Q", "R", "S", "T", "V", "W", "Y"]
df_structure = pd.DataFrame(df_structure_no_mt.values.repeat(20, axis=0), columns=df_structure_no_mt.columns)
for i in range(0, len(df_structure), 20):
    for j in range(20):
        df_structure.iloc[i+j, :]["variant"] = df_structure.iloc[i+j, :]["variant"][:-1] + aa_list[j]
df_structure.drop(columns="index", inplace=True)

# Load PDB amino acid frequencies used to approximate unfolded states
pdb_nlfs = -np.log(np.load(f"{os.getcwd()}/data/pdb_frequencies.npz")["frequencies"])

# # Add wt and mt idxs and freqs to df

df_structure["wt_idx"] = df_structure.apply(lambda row: one_to_index(row["variant"][0]), axis=1)
df_structure["mt_idx"] = df_structure.apply(lambda row: one_to_index(row["variant"][-1]), axis=1)
df_structure["wt_nlf"] = df_structure.apply(lambda row: pdb_nlfs[row["wt_idx"]], axis=1)
df_structure["mt_nlf"] = df_structure.apply(lambda row: pdb_nlfs[row["mt_idx"]], axis=1)

# Define models
best_cavity_model_path = open(f"/content/output/cavity_models/best_model_path.txt", "r").read()
cavity_model_net = CavityModel(get_latent=True).to(DEVICE)
cavity_model_net.load_state_dict(torch.load(f"{best_cavity_model_path}"))
cavity_model_net.eval()
ds_model_net = DownstreamModel().to(DEVICE)
ds_model_net.apply(init_lin_weights)
ds_model_net.eval()

###set start time
start_time = time.perf_counter()

# Make ML predictions
print(f"Starting downstream model prediction")
dataset_key="predictions"
df_ml = ds_pred(cavity_model_net,
                ds_model_net,
                df_structure,
                dataset_key,
                NUM_ENSEMBLE,
                DEVICE,
                ) 
print(f"Finished downstream model prediction")
end_time = time.perf_counter()
elapsed = datetime.timedelta(seconds = end_time - start_time)
print("Complete - prediction execution took", elapsed)

elapsed = datetime.timedelta(seconds = end_time - start_time)
print("Generating output files")
#Merge and save data with predictions

df_total = df_structure.merge(df_ml, on=['pdbid','chainid','variant'], how='outer')
#df_total["b_factors"] = df_total.apply(lambda row: row["resenv"].b_factors, axis=1)
df_total = df_total.drop("resenv", 1)
print(f"{len(df_structure)-len(df_ml)} data points dropped when matching total data with ml predictions in: {dataset_key}.")

# Parse output into separate files by pdb, print to PRISM format
for pdbid in df_total["pdbid"].unique():
    df_pdb = df_total[df_total["pdbid"]==pdbid]
    for chainid in df_pdb["chainid"].unique():
        pred_outfile = f"{os.getcwd()}/output/{dataset_key}/cavity_pred_{pdbid}_{chainid}.csv"
        print(f"Parsing predictions from pdb: {pdbid}{chainid} into {pred_outfile}")
        df_chain = df_pdb[df_pdb["chainid"]==chainid]
        df_chain = df_chain.assign(pos = df_chain["variant"].str[1:-1])
        df_chain['pos'] = pd.to_numeric(df_chain['pos'])
        first_res_no = min(df_chain["pos"])
        df_chain = df_chain.assign(wt_AA = df_chain["variant"].str[0])
        df_chain = df_chain.assign(mt_AA = df_chain["variant"].str[-1])
        seq = get_seq_from_variant(df_chain)
        df_chain.to_csv(pred_outfile, index=False)
        prism_outfile = f"/content/output/{dataset_key}/prism_cavity_{pdbid}_{chainid}.txt"

        # if (AF_ID !=''):
        #   prism_outfile = f"/content/output/{dataset_key}/prism_cavity_{AF_ID}_{chainid}.txt"
        # elif (PDB_ID !=''):
        #   prism_outfile = f"/content/output/{dataset_key}/prism_cavity_{PDB_ID}_{chainid}.txt"
        # elif PDB_custom:
        #   prism_outfile = f"/content/output/{dataset_key}/prism_cavity_XXXX_{chainid}.txt"
        cavity_to_prism(df_chain, pdbid, chainid, seq, prism_outfile)

# End timer and print result
#!rm /content/output/predictions/*xxxx*.csv
elapsed = datetime.timedelta(seconds = end_time - start_time)
print("Complete - files generated")

#@markdown ****

In [ ]:
#@title <b><font color='#56b4e9'> Download results as archive </font></b>

#@markdown Run the cell to <b><font color='#009e74'> download a .zip archive </font></b> with prediction files for the <ins>current run</ins>.

#@markdown <ins>Tick</ins> the next box if you ran multiple predictions and you want to <ins>download all of them</ins>.

download_all_predictions= False #@param {type:"boolean"}

if download_all_predictions:
  os.system( "zip -r {} {}".format( f"predictions_output_all.zip" , f"/content/output/predictions/*" ) )
  files.download(f"predictions_output_all.zip")
else:
  if (AF_ID !=''):
    os.system( "zip -r {} {}".format( f"predictions_output_{AF_ID}.zip" , f"/content/output/predictions/*{AF_ID}*" ) )
    files.download(f"predictions_output_{AF_ID}.zip")
  elif (PDB_ID !=''):
    os.system( "zip -r {} {}".format( f"predictions_output_{PDB_ID}.zip" , f"/content/output/predictions/*{PDB_ID}*" ) )
    files.download(f"predictions_output_{PDB_ID}.zip")
  else:
    os.system( "zip -r {} {}".format( f"predictions_output.zip" , f"/content/output/predictions" ) )
    files.download(f"predictions_output.zip")

  if download_all_predictions:
    os.system( "zip -r {} {}".format( f"predictions_output.zip" , f"/content/output/predictions" ) )
    files.download(f"predictions_output_all.zip")

#@markdown **P.S.: prediction files are also stored in the colab file system folder: `/output/predictions/`**
#@markdown ****

**Troubleshooting**

- Check that the runtime type is set to GPU at "Runtime" -> "Change runtime type".
- Try to restart the session "Runtime" -> "Factory reset runtime".
- Check your input pdb.

\\

**Known problems:**

- Residues with numeration index below 0 are not supported by the output file parser and thus they deleted from the pdb in the pre-processing step.
- Insertion annotations in the pdb are not supported. Any annotations is actually deleted during the pre-processing step.

\\

**License:**

RaSP's source code is licensed under the permissive Apache Licence, Version 2.0.
 Additionally, this notebook uses the reduce source code which license could be find in `/content/src/pdb_parser_scripts/reduce/`

\\

**Bugs:**

For any bugs please report the issue on the project [Github](https://github.com/KULL-Centre/papers/tree/main/2022/ML-ddG-Blaabjerg-et-al) or contact one of the listed authors in the connected [manuscript](https://www.biorxiv.org/content/10.1101/2022.07.14.500157v1).

\\

**Citing this work:**

If you use our model please cite:

Blaabjerg, L.M., Kassem, M.M., Good, L.L., Jonsson, N., Cagiada, M., Johansson, K.E., Boomsma, W., Stein, A. and Lindorff-Larsen, K., 2022. *Rapid protein stability prediction using deep learning representations*, bioRxiv. (https://doi.org/10.1101/2022.07.14.500157)



```
@article{blaabjerg2022rapid,
  title={Rapid protein stability prediction using deep learning representations},
  author={Blaabjerg, Lasse M and Kassem, Maher M and Good, Lydia L and Jonsson, Nicolas and Cagiada, Matteo and Johansson, Kristoffer E and Boomsma, Wouter and Stein, Amelie and Lindorff-Larsen, Kresten},
  journal={bioRxiv},
  year={2022},
  publisher={Cold Spring Harbor Laboratory}
}
```

